In [1]:
import sys
sys.path.append('/data/spark15/python/')
import numpy as np
import pandas as pd

import json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.types import *

In [2]:
import tweepy

In [3]:
sc = SparkContext("local", "w205_2017_fall_final_project")
sqlContext = SQLContext(sc)
hc = HiveContext(sc)

### How to store a table

In [34]:
df = pd.read_json('twitter_store.json', lines=True)
df = df.assign(user_location = df.user.apply(lambda d: d['location']))

In [42]:
df.head() # our raw twitter data from json file

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,user_location
0,NaN,None,2017-11-13 01:19:16,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,"{u'display_text_range': [0, 166], u'entities':...",0,False,low,...,0,0,False,NaN,"<a href=""http://share.radionomy.com"" rel=""nofo...","James Bong ""James Bong / Mr. Kiss Kiss Bong Bo...",2017-11-13 01:19:16.166,True,"{u'follow_request_sent': None, u'profile_use_b...","Oak Park, IL"
1,NaN,None,2017-11-13 01:19:16,NaN,"{u'user_mentions': [{u'id': 140565980, u'indic...",NaN,NaN,0,False,low,...,0,0,False,NaN,"<a href=""https://www.fastcast4u.com"" rel=""nofo...",#NowPlaying Spirit 2 Spirit @messagemusic1 - W...,2017-11-13 01:19:16.225,False,"{u'follow_request_sent': None, u'profile_use_b...","Winter Park, FL"
2,NaN,None,2017-11-13 01:19:16,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,0,0,False,NaN,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",心の薬/乃木坂46 #nowplaying #乃木坂46#nogizaka46,2017-11-13 01:19:16.984,False,"{u'follow_request_sent': None, u'profile_use_b...",None
3,NaN,None,2017-11-13 01:19:17,NaN,"{u'user_mentions': [{u'id': 200204390, u'indic...",NaN,NaN,0,False,low,...,0,0,False,NaN,"<a href=""https://www.fastcast4u.com"" rel=""nofo...",#NowPlaying Conceptz - Get Back To Ya ft. Stri...,2017-11-13 01:19:17.393,False,"{u'follow_request_sent': None, u'profile_use_b...","New York, USA"
4,NaN,None,2017-11-13 01:19:17,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,0,0,False,NaN,"<a href=""http://twt.googlecode.com"" rel=""nofol...",#NowPlaying Love Me Now by John Legend on #Urb...,2017-11-13 01:19:17.437,False,"{u'follow_request_sent': None, u'profile_use_b...",United States


In [37]:
table = df[['text', 'timestamp_ms', 'coordinates', 'id', 'user_location']].set_index('id')

In [41]:
table.head() # refined into the table we would like to store

,text,timestamp_ms,coordinates,user_location
id,,,,
929881313548517376,"James Bong ""James Bong / Mr. Kiss Kiss Bong Bo...",2017-11-13 01:19:16.166,None,"Oak Park, IL"
929881313795911680,#NowPlaying Spirit 2 Spirit @messagemusic1 - W...,2017-11-13 01:19:16.225,None,"Winter Park, FL"
929881316979458048,心の薬/乃木坂46 #nowplaying #乃木坂46#nogizaka46,2017-11-13 01:19:16.984,None,None
929881318695014400,#NowPlaying Conceptz - Get Back To Ya ft. Stri...,2017-11-13 01:19:17.393,None,"New York, USA"
929881318879502336,#NowPlaying Love Me Now by John Legend on #Urb...,2017-11-13 01:19:17.437,None,United States


In [39]:
spark_df = hc.createDataFrame(table)
spark_df.write.saveAsTable('tweet_table')

### How to pull a table and interact with spark-sql

In [40]:
hc.sql('select * from tweet_table').toPandas()

,text,timestamp_ms,coordinates,user_location
0,"James Bong ""James Bong / Mr. Kiss Kiss Bong Bo...",1510535956166000000,None,"Oak Park, IL"
1,#NowPlaying Spirit 2 Spirit @messagemusic1 - W...,1510535956225000000,None,"Winter Park, FL"
2,心の薬/乃木坂46 #nowplaying #乃木坂46#nogizaka46,1510535956984000000,None,None
3,#NowPlaying Conceptz - Get Back To Ya ft. Stri...,1510535957393000000,None,"New York, USA"
4,#NowPlaying Love Me Now by John Legend on #Urb...,1510535957437000000,None,United States
5,#NowPlaying on @teez_fm @VVbrown : Sacrifice\n...,1510535958237000000,None,None
6,#radiokpopway #NowPlaying DAY6 - I Wait - List...,1510535958666000000,None,RLA Selective
7,#NowPlaying Thrillseekers (Chuck Daniels Remix...,1510535958886000000,None,Europe
8,Tye Tribbett - Live https://t.co/U7MzqrGrUR #n...,1510535959009000000,None,"Jacksonville, FL"
9,#NowPlaying Culture Club - Love Is Love #liste...,1510535959427000000,None,Argentina
